In [ ]:
import os, sys
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.info("Application started successfully.")

In [ ]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "etl"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
# create bronze silver in nessie catalog
spark.sql(f"USE REFERENCE main IN nessie").show()
namespace = "SILVER"
df = spark.sql("SHOW NAMESPACES in nessie")
namespace_exists = df.filter(df.namespace == namespace).count() > 0

logger.info(f"namespace {namespace} exists: {namespace_exists}")
if not namespace_exists:
    spark.sql(f"CREATE NAMESPACE nessie.{namespace}").show()

spark.sql("SHOW NAMESPACES in nessie").show()


In [ ]:
# create etl branch
spark.sql(f"USE REFERENCE main IN nessie").show()
spark.sql(f"DROP BRANCH IF EXISTS {REF}").show()
spark.sql(f"CREATE BRANCH {REF} IN nessie FROM main").show()
spark.sql(f"LIST REFERENCES IN nessie").show()

In [ ]:
import notebooks.lib.silver_fact_adventureWorks_sql as sql
import importlib
importlib.reload(sql)

source_list = [
    {
        "table_name": "SALES_HEADER",
        "sql_create_table": sql.sql_create_silver_sales_header,
        "sql_select": sql.sql_select_bronze_sales_header,
        "enabled": True
    },
    {
        "table_name": "SALES_DETAIL",
        "sql_create_table": sql.sql_create_silver_sales_detail,
        "sql_select": sql.sql_select_bronze_sales_detail,
        "enabled": True
    },
    {
        "table_name": "CURRENCY_RATE_HISTORY",
        "sql_create_table": sql.sql_create_silver_currency_rate_history,
        "sql_select": sql.sql_select_bronze_currency_rate_history,
        "enabled": True
    }
]

In [ ]:
import pyspark.sql.functions as sqlf

spark.sql(f"USE REFERENCE {REF} IN nessie").show()

for item in [item for item in source_list if item["enabled"]]:
    table_name = item["table_name"].upper()

    # check table if exists
    df_table_list = spark.sql(f"SHOW TABLES IN nessie.{namespace}")
    df_table_list.withColumn("tableName", sqlf.concat_ws("_", sqlf.lit(namespace), sqlf.col("tableName")))
    df_table_list.show()
    table_exists = df_table_list.filter(df_table_list.tableName == table_name).count() > 0  
    
    if table_exists:
        # truncate table
        logger.info(f'dropping table: {table_name}')
        spark.sql(f"DROP TABLE {namespace}.{table_name}")
    
    # create table
    logger.info(f'creating table if not exists:{table_name}')
    spark.sql(item["sql_create_table"])

    # select data from bronze table
    logger.info('selecting data from bronze')
    df = spark.sql(item["sql_select"])
    num_rows, num_columns = df.count(), len(df.columns)

    # insert data into silver
    logger.info(f'inserting data ({num_rows} rows {num_columns}, columns) {table_name}')
    df.writeTo(f"{namespace}.{table_name}").append()
    logger.info(f'data inserted successfully')



In [ ]:
spark.sql("LIST REFERENCES IN nessie").show()

In [ ]:
spark.sql(f"MERGE BRANCH {REF} INTO main IN nessie").show()
spark.sql(f"DROP BRANCH {REF} IN nessie").show()

In [ ]:
spark.sql("LIST REFERENCES IN nessie").show()